---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [4]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [5]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [6]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    import pandas as pd 
    
    ut_list=[]
    with open('university_towns.txt') as f:
        for line in f:
            if '[edit]'in line:
                state = line.split('[')[0].strip()
            else:
                ut_list.append((state,line.split('(')[0].strip()))

    university_towns=pd.DataFrame(ut_list)
    university_towns.columns=['State','RegionName']
    

    return university_towns

In [7]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    import pandas as pd
    
    df=pd.read_excel('gdplev.xls',skiprows=7)
    #GDP_year=df[df.columns[[0,1,2]]].dropna()
    #GDP_year.columns=['Year','current dollars','chained 2009 dollars']
    GDP=df[df.columns[[4,5,6]]].dropna()
    GDP.columns=['Quarter','current dollars','chained 2009 dollars']
    GDP['year']=GDP['Quarter'].apply(lambda x: x.split('q')[0].strip())
    GDP=GDP[GDP['year'].apply(lambda x: int(x))>=2000].reset_index(drop=True)
    
    year_quarter_start=[]
    
    for i in range(0,len(GDP)-2):
        if (GDP.iloc[i][1] > GDP.iloc[i+1][1]) & (GDP.iloc[i+1][1] > GDP.iloc[i+2][1]):
            year_quarter_start.append(GDP.iloc[i][0])
        
    return year_quarter_start[0]

In [8]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    import pandas as pd 
    start = get_recession_start()
    df=pd.read_excel('gdplev.xls',skiprows=7)
    GDP=df[df.columns[[4,5,6]]].dropna()
    GDP.columns=['Quarter','current dollars','chained 2009 dollars']
    GDP['year']=GDP['Quarter'].apply(lambda x: x.split('q')[0].strip())
    GDP=GDP[GDP['year'].apply(lambda x: int(x))>=2000].reset_index(drop=True)
    start_index = GDP[GDP['Quarter']==start].index.values
    year_quarter_end = []
    
    for i in range(start_index[0],len(GDP)-2):
        if (GDP.iloc[i][1] < GDP.iloc[i+1][1]) & (GDP.iloc[i+1][1] < GDP.iloc[i+2][1]):
            year_quarter_end.append(GDP.iloc[i+2][0])
    
    return year_quarter_end[0]

In [9]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    start = get_recession_start()
    df=pd.read_excel('gdplev.xls',skiprows=7)
    GDP=df[df.columns[[4,5,6]]].dropna()
    GDP.columns=['Quarter','current dollars','chained 2009 dollars']
    GDP['year']=GDP['Quarter'].apply(lambda x: x.split('q')[0].strip())
    GDP=GDP[GDP['year'].apply(lambda x: int(x))>=2000].reset_index(drop=True)
    
    start_index = GDP[GDP['Quarter']==start].index.values
    end = get_recession_end()
    end_index = GDP[GDP['Quarter']==end].index.values
    recession=GDP.iloc[start_index[0]:end_index[0]+1].reset_index(drop=True)
    recession_bottom=recession.iloc[recession['current dollars'].argmin()][0]
    
    return recession_bottom

In [13]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    '''import pandas as pd
    house_data=pd.read_csv('City_Zhvi_AllHomes.csv')
    year_start_index=house_data.columns.get_loc('2000-01')
    year_month=house_data.iloc[:,51:]
    
    for i in range(year_month.shape[1]):
        year=year_month.columns[i].split('-')[0]
        if year_month.columns[i].split('-')[1].strip() in ['01','02','03']:
            name=year+'q1'
            year_month.rename(columns={year_month.columns[i]:name},inplace=True)
        elif year_month.columns[i].split('-')[1].strip() in ['04','05','06']:
            name=year+'q2'
            year_month.rename(columns={year_month.columns[i]:name},inplace=True)
        elif year_month.columns[i].split('-')[1].strip() in ['07','08','09']:
            name=year+'q3'
            year_month.rename(columns={year_month.columns[i]:name},inplace=True)
        else:
            name=year+'q4'
            year_month.rename(columns={year_month.columns[i]:name},inplace=True)
    
    quarter_data=pd.merge(house_data[['State','RegionName']],year_month.transpose().groupby(level=0).mean().transpose(),left_index=True,right_index=True)
    
    quarter_data.set_index(['State','RegionName'],inplace=True)
    '''
    
    import pandas as pd
    house_data=pd.read_csv('City_Zhvi_AllHomes.csv')
    house_data['State']=house_data['State'].map(states)
    house_data.set_index(['State','RegionName'],inplace=True)
    house_data=house_data.loc[:,'2000-01':]
    
    new_columns = [str(x)+y for x in range(2000,2016) for y in ['q1','q2','q3']]
    new_columns = new_columns[:-1]
    
    x = 0
    
    for column in new_columns: 
        house_data[column] = house_data.iloc[:,x:x+3].mean(axis=1)
        x += 3
        
    house_data=house_data.loc[:,'2000q1':]
    
    return house_data

In [26]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    import pandas as pd
    from scipy import stats
    
    start = get_recession_start()
    #get the bottom of the recession
    bottom = get_recession_bottom()
    # get the zillow housing data in desired format
    housing_data = convert_housing_data_to_quarters()
    # keep only the columns from recession starting point and bottom.
    housing_data = housing_data.loc[:, start: bottom]
    # compute price ratio between the recession points and add it as a column.
    housing_data.reset_index(inplace=True)
    housing_data['price_ratio'] = (housing_data[start] - housing_data[bottom]) / housing_data[start]
    # get the university town list to split the data.
    uni_towns = get_list_of_university_towns()  
    uni_town_list = uni_towns['RegionName'].tolist()
    # add a column to use as splitting condition
    housing_data['isUniTown'] = housing_data.RegionName.apply(lambda x: x in uni_town_list)
    #split the data into two separate dataframes and drop rows with missing values. The dropping step is needed to
    #perform the t-test
    uni_data = housing_data[housing_data.isUniTown].copy().dropna()
    not_uni_data = housing_data[~housing_data.isUniTown].copy().dropna()
    # get the p-value by applying t-test on these two dataframe columns.
    p = ttest_ind(uni_data['price_ratio'], not_uni_data['price_ratio'])[1]
    # this boolean value will tell us whether we can reject the null hyopthesis or not.
    different = p < 0.01   
    # this metric will tell us which type of town has the lower housing price ratio (mean) during the recession
    lower = 'university town' if uni_data['price_ratio'].mean() < not_uni_data['price_ratio'].mean() else 'non-university town'
    


    
    return (different, p, lower)